In [258]:
def get_config():
    return {
        'x': 'expanded!',
        'var': {'h':'hello {testvar} expansion'},
        'a': { 'a':1, 'b':2, 'c':3 },
        'b': { 'a':10, 'b':20, 'c':30 },
        'deep': { 'a': {'a':100, 'b':200, 'c':300} },
        'var2': '{testvar} shallow'
    }

def load_config_alpha(config_obj, *field_names):
    pass

In [259]:
from collections import UserDict

class MyDict(UserDict):
    pass

conf = get_config()
a = MyDict(conf)
isinstance(a, dict), isinstance(conf, dict), isinstance(conf, MyDict), isinstance(conf, UserDict), isinstance(a, UserDict)

(False, True, False, False, True)

In [260]:
from collections import UserDict

class ConfigSection(UserDict):
    """
    A dictionary wrapper that implements some useful features:
    - introspection: a section knows its name and key in the parent section
    - deep (recursive) cast from dict.  
      That is, you can create a Config with Configs from dict with dicts.
    
    Note that isinstance(userDict, dict) == false
    """

    def __init__(self, _dict=None, name=''):
        UserDict.__init__(self, _dict)

        self.name = name
        
        for key in self.data:
            val = self.data[key]
            if isinstance(val, dict):
                # deep convert dict to config
                if self.name:
                    childname = self.name+'.'+key
                else:
                    childname = key
                self.data[key] = ConfigSection(val, childname)
    
    def getp(self, path: str):
        """
        A getter function for dot-separated field paths.
        For instance, `config.getp('a.b.c') == config['a']['b']['c']`.

        Can return both config sections and values. 
        """
        keyseq = path.strip('.').split('.')

        if keyseq[0] not in self.data:
            raise KeyError(f"ConfigSection '{self.name}' has no '{keyseq[0]}' section")

        section = self
        breadcrumbs = self.name
        for nextkey in keyseq:
            if isinstance(section, ConfigSection):
                if nextkey in section:
                    section = section[nextkey]
                    breadcrumbs += '.' + nextkey
                else:
                    raise KeyError(f"{path} : '{breadcrumbs}' section has no '{nextkey}' field")
            else:
                raise KeyError(f"{path} : {breadcrumbs + '.' + nextkey} is a field, not a section")
        return section

    def to_flat_dict(self):
        """
        Returns a dict with keys in form of 'key1.key2.key3'.
        `section.to_flat_dict()['a.b.c'] == section['a']['b']['c']`
        """
        raise NotImplementedError()

class Config(ConfigSection):
    """
    Another dictionary wrapper. Config manages a tree of ConfigSections.
    Features:
    - string variable expansion
    - yaml loading/writing
    """
    import re
    
    config_var_def = {
        'tmpdir': 'general.tmp_dir',
        'anndir': 'general.annotation_outdir_local',
        'mtdir': 'general.matrixtables_outdir',
        'testvar': 'x',
    }

    __path_field_re = re.compile(r".*_[out|in]?[dir|file]")
    __cvar_re = re.compile(r"\{[a-zA-Z0-9_]+\}")
    SKIP_EXPAND = False
    EXPAND_ALL = True

    @staticmethod
    def _should_expand(fieldname: str):
        return Config.EXPAND_ALL or (
            not Config.SKIP_EXPAND and Config.__path_field_re.fullmatch(fieldname) is not None
        )

    def _expand_field(self, val: str):
        config_vars = {}
        required_vars = [x[1:-1] for x in Config.__cvar_re.findall(val)]
        for k in required_vars:
            if k in Config.config_var_def:
                config_vars[k] = self.getp(Config.config_var_def[k])
            else:
                config_vars[k] = '{'+k+'}'
        return val.format(**config_vars)

    def _expand_all_fields(self, section: ConfigSection):
        for key in section.data:
            val = section.data[key]
            if isinstance(val, str) and Config._should_expand(key):
                section.data[key] = self._expand_field(val)
            elif isinstance(val, ConfigSection):
                self._expand_all_fields(val)


    def getp(self, path: str):
        """
        A getter function for dot-separated field paths.
        For instance, `config.getp('a.b.c') == config['a']['b']['c']`.

        Can return both config sections and values. 
        """
        keyseq = path.strip('.').split('.', 1)
        fk = keyseq[0]  # first key
        if fk not in self.data:
            raise KeyError(f"Config has no '{fk}' section")
        if len(keyseq) > 1 and not isinstance(self.data[fk], ConfigSection):
            raise KeyError(f"{path} : {fk} is a field, not a section")
        if len(keyseq) == 1:
            return self.data[fk]
        remaining_path = keyseq[1]
        return self.data[fk].getp(remaining_path)

    def __init__(self, _dict=None):
        ConfigSection.__init__(self, _dict, '')
        self._expand_all_fields(self)



In [261]:
conf = Config(get_config())
conf

{'x': 'expanded!', 'var': {'h': 'hello expanded! expansion'}, 'a': {'a': 1, 'b': 2, 'c': 3}, 'b': {'a': 10, 'b': 20, 'c': 30}, 'deep': {'a': {'a': 100, 'b': 200, 'c': 300}}, 'var2': 'expanded! shallow'}

In [262]:
type(conf.getp('a'))

__main__.ConfigSection

In [263]:
conf._expand_field('{testvar}')

'expanded!'

In [266]:
conf.getp('deep.a')

{'a': 100, 'b': 200, 'c': 300}

---

In [318]:

def getp(_dict: dict, keypath: str, silent: bool = False, default = None):
    """
    Get a value from a tree of dicts using dot-notation. That is, 
    `getp(conf, "a.b.c") == conf['a']['b']['c']`

    - `_dict` is a structure of nested dicts
    - `keypath` is a sequence of keys joined by '.'
    - if `silent` then return `default` value on invalid path instead of raising an exception  

    Note that this function can return a dict as well as a scalar -- it has no output type checks.
    """
    keyseq = keypath.strip('.').split('.')
    if keyseq[0] not in _dict:
        if silent: return default
        raise KeyError(f"top-level dict has no '{keyseq[0]}' key")
    
    section = _dict
    breadcrumbs = ''
    for nextkey in keyseq:
        if isinstance(section, dict):
            if nextkey in section:
                section = section[nextkey]
                breadcrumbs += '.' + nextkey
            else:
                if silent: return default
                raise KeyError(f"{keypath} : '{breadcrumbs}' section has no '{nextkey}' field")
        else:
            if silent: return default
            raise KeyError(f"{keypath} : {breadcrumbs + '.' + nextkey} is a field, not a section")
    return section


In [319]:

getp(get_config(), 'deep.a.b')

200

In [320]:
import re
cvar_re = re.compile(r"\{([a-zA-Z0-9_]+(?:\.[a-zA-Z0-9_]+)*)\}")

In [321]:
cvar_re.findall('hey{hello}, {foo.bar}/{.omg.hiii}')

['hello', 'foo.bar']

---

In [322]:

def expand_cvars(config: dict, str_with_cvar: str):
    """
    Expand config variables in a string.  
    Ignore errors, leave invalid cvars as is.
    """
    import re
    # Find all cvars that look like {cvar_name}
    # - no dots at the begininng or at the end
    # - no more 1 dot in a row
    # - a-zA-Z0-9_ as identifiers
    cvar_re = re.compile(r"\{([a-zA-Z0-9_]+(?:\.[a-zA-Z0-9_]+)*)\}")
    def repl(cvar_match):
        cvar_name = cvar_match.group(1)
        return str(getp(config, keypath=cvar_name, silent=True, default='{'+cvar_name+'}'))
    expanded_str = cvar_re.sub(repl, str_with_cvar) 
    return expanded_str

In [325]:
expand_cvars(get_config(), 'omg hiii {x} and {deep.a.b} and even {do.not.exists}')

'omg hiii expanded! and 200 and even {do.not.exists}'

---

In [329]:

def expand_cvars_recursive(config: dict, dict_to_expand, inplace=False):
    """
    Recursively expand cvars in all string fields in a nested dict structure `dict_to_expand`
    using data from `config`
    """
    if inplace:
        _dict = dict_to_expand
    else:
        from copy import deepcopy
        _dict = deepcopy(dict_to_expand)
    for key in _dict:
        val = _dict[key]
        if isinstance(val, str):
            _dict[key] = expand_cvars(config, val)
        elif isinstance(val, dict):
            expand_cvars_recursive(config, val, inplace=True)
    return _dict


In [330]:
conf = {
    'a': 10,
    'b': 20,
    'c': 30,
    'deep' : {
        'a': 100,
        'b': 200,
        'c': 300
    }
}
dat = {
    'foo' : 'hello ten {a} and hundred {deep.a}',
    'bar' : {
        'baz': 'hey there from the deep {deep.c}',
        'lol': 'shallow this time {c}',
        'no': 'no expansion needed',
        'num': 123
    },
    'num': 456
}

expand_cvars_recursive(conf, dat)

{'foo': 'hello ten 10 and hundred 100',
 'bar': {'baz': 'hey there from the deep 300',
  'lol': 'shallow this time 30',
  'no': 'no expansion needed',
  'num': 123},
 'num': 456}

In [331]:
dat

{'foo': 'hello ten {a} and hundred {deep.a}',
 'bar': {'baz': 'hey there from the deep {deep.c}',
  'lol': 'shallow this time {c}',
  'no': 'no expansion needed',
  'num': 123},
 'num': 456}

In [333]:
a = expand_cvars_recursive(conf, dat, inplace=True)
a == dat

True

In [334]:
a

{'foo': 'hello ten 10 and hundred 100',
 'bar': {'baz': 'hey there from the deep 300',
  'lol': 'shallow this time 30',
  'no': 'no expansion needed',
  'num': 123},
 'num': 456}

---

In [364]:

__is_path_field_re = re.compile(r"(out|in)?(dir|file)(_local)?$")

def is_path_field(fieldname):
    return __is_path_field_re.search(fieldname) is not None

In [363]:
__is_path_field_re.search('test.fii.asd.resource_outdir_local').groups()

('out', 'dir', '_local')

---

In [365]:
path_cvar_re = re.compile(r"(\/|\\)?\{([a-zA-Z0-9_]+(?:\.[a-zA-Z0-9_]+)*)\}(\/|\\)?")
pr = 'foo/{r}'
pl = '{l}/bar'
plr = 'foo/{lr}/bar'

conf = {
    'l': '/leftdir/',
    'r': '/rightdir',
    'rl': '/middledir/',
}

In [371]:
path_cvar_re.search(plr).groups()

('/', 'lr', '/')

In [391]:
import pathlib
path_with_cvars = pathlib.Path('file:///usr/plots//plot.1.jpg')
path_with_cvars.parts

('file:', 'usr', 'plots', 'plot.1.jpg')

In [392]:
import posixpath 
posixpath.normpath('file:///usr/plots//plot.1.jpg')

'file:/usr/plots/plot.1.jpg'

In [403]:
protocol_re = re.compile(r"[a-zA-Z][a-zA-Z0-9]*:\/\/")
protocol_needed = protocol_re.match('file:///usr/plots//plot.1.jpg')
protocol_needed.end()

7